## Project 2: Thermoelectric Generator

Import and Install Libraries

In [1]:
%pip install pandas pyfluids sympy scipy numpy matplotlib openpyxl

import pandas as pd # This makes nice tables
import pyfluids # This does the thermodynamics
import math
import scipy.optimize
from sympy import symbols, Eq, solve
import numpy as np
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\obrau\AppData\Local\Temp\ipykernel_27032\429236754.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # This makes nice tables


Power Requirements

In [2]:
power_data = [
    ["Raspberry Pi Pico", 42, 0.1386],
    ["Lora Radio Module", 125, 0.4125],
    ["pH Sensor", 13.3, 0.04389],
    ["Salinity", 34, 0.1122],
]
power_draw = pd.DataFrame(power_data, columns=["Component", "Peak Amperage Draw (mA)", "Power Draw (W)"])

power_draw

,Component,Peak Amperage Draw (mA),Power Draw (W)
0,Raspberry Pi Pico,42.0,0.13860
1,Lora Radio Module,125.0,0.41250
2,pH Sensor,13.3,0.04389
3,Salinity,34.0,0.11220


Temperature Data

In [3]:
minT_data = [
    [1,24],
    [2,25],
    [3,26],
    [4,21],
]
min_Temp = pd.DataFrame(minT_data, columns=["Month", "Min Average Temp (C)"])

min_Temp

,Month,Min Average Temp (C)
0,1,24
1,2,25
2,3,26
3,4,21


In [4]:
maxT_data = [
    [1,32],
    [2,36],
    [3,37],
    [4,32],
]
max_Temp = pd.DataFrame(maxT_data, columns=["Month", "Max Average Temp (C)"])

max_Temp

,Month,Max Average Temp (C)
0,1,32
1,2,36
2,3,37
3,4,32


Solar Irradiation

In [5]:
irradiation_data = [
    ["Min",4.38],
    ["Max",4.93],
]
irradiation = pd.DataFrame(irradiation_data, columns=["Solar Irradiation", "Daily Output (kWh/m^2)"])

irradiation

,Solar Irradiation,Daily Output (kWh/m^2)
0,Min,4.38
1,Max,4.93


Wind Speed

In [6]:
wind_Data = [[3]]
wind_Speed = pd.DataFrame(wind_Data,columns = ["Average Wind Speed (m/s)"])
wind_Speed

,Average Wind Speed (m/s)
0,3


Hourly Constants

In [7]:
# Load the Excel file into a pandas DataFrame
constant_frame = pd.read_excel('HourlyConstants.xlsx', header=1)

print(constant_frame.columns)

# Now you can work with the DataFrame 'df'
print(constant_frame.head())  # Display the first few rows of the DataFrame?

Index(['Month', 'Day', 'Hour', 'Beam Irradiance (W/m2)',
       'Diffuse Irradiance (W/m2)', 'Ambient Temperature (C)',
       'Wind Speed (m/s)'],
      dtype='object')
   Month  Day  Hour  Beam Irradiance (W/m2)  Diffuse Irradiance (W/m2)  \
0      6    1     0                       0                          0   
1      6    1     1                       0                          0   
2      6    1     2                       0                          0   
3      6    1     3                       0                          0   
4      6    1     4                       0                          0   

   Ambient Temperature (C)  Wind Speed (m/s)  
0                       21               0.8  
1                       21               0.8  
2                       20               0.7  
3                       20               0.7  
4                       20               0.7  


Heat Transfer Characteristics

In [8]:
heat_data = [
    ["Number of Units",21,"3 sets of 7 in series"],
    ["Unit Length",0.03,"m"],
    ["Unit Thickness",3.75E-3,"m"],
    ["Thermal Conductivity",1.72,"W/mK"],
    ["Emissivity",.9,"Painted Black"],
    ["Absorptivity",.9,"Painted Black"],
    ["Assumed Water Temperature",30,"Degree C"],
]
heat_parameters = pd.DataFrame(heat_data, columns=["Parameter", "Value","Units/Notes"])

heat_parameters

,Parameter,Value,Units/Notes
0,Number of Units,21.00000,3 sets of 7 in series
1,Unit Length,0.03000,m
2,Unit Thickness,0.00375,m
3,Thermal Conductivity,1.72000,W/mK
4,Emissivity,0.90000,Painted Black
5,Absorptivity,0.90000,Painted Black
6,Assumed Water Temperature,30.00000,Degree C


In [9]:
from pyfluids import Fluid, FluidsList, Input

water = Fluid(FluidsList.Water)

water_ambient = water.with_state(
    Input.pressure(101325), Input.temperature(30)
)
#print(water_ambient.density)

Natural Convection for the Bottom of the Plate

In [10]:
def calcBeta(Tc,Tw):

    water_Amb = water.with_state(
    Input.pressure(101325), Input.temperature(Tw-273)
    )

    rhoAmb= water_Amb.density

    T = Tc - 273
    rho = (999.83952 + 16.945176 * T - 7.9870401E-3 * (T**2) - 49.170461E-6 * (T**3) + 105.56302E-9 * (T**4) - 280.54253E-12 * (T**5))/(1+16.897850E-3 * T)
    print(rho)

    beta = -1/rho * (rhoAmb-rho)/(Tw-Tc)
    return beta


In [11]:
def calcRalyeighNumber(beta, Tc, Tw, x, alpha):
    #Find dT
    deltaT = Tc - Tw
    #Define Constants
    g = 9.81
    kinematic_viscosity = 8.005E-7
    #Calculate Ra
    Rax  = abs(g * beta * deltaT * (x)**3 / (kinematic_viscosity * alpha))
    return Rax

In [12]:
def bottomConvection(Tc,Tw,L,alpha):
    Pr = 6.9
    beta = calcBeta(Tc,Tw)
    
    #Calculate Ralyeigh Number at x = L
    RaL = calcRalyeighNumber(beta,Tc, Tw, L, alpha)

    #Critical Rayleigh Number
    if RaL < 1E9:
        NuL = 0.68 + 0.670 * (RaL) ** (1/4) / (( 1- (0.492/Pr) ** (9/16)) ** (4/9))
    else:
        NuL = (0.825+ 0.397 * (RaL) ** (1/6) / (( 1 + (0.492/Pr) ** (9/16)) ** (8/27))**2)

    kf = 0.598E-3 # kW/mK


    return NuL



Forced Convection on Top of the Plate

In [13]:
def topForcedConvectionCoefficient(u,Lplate):
    #Forced Convection on the top of the plate
    #Constants for 
    kf = 27.06 #kW/(mk)
    kinematic_viscosity = 1.61E-5
    Pr = 0.71

    Re = (u**2)  * Lplate / (kinematic_viscosity)

    NuL = 0.453 * (Re ** (1/2)) * (Pr ** 1/3)
    ha = NuL * kf / Lplate

    return ha


Heat Balance Equations

In [14]:
def updatedSolution(u,Lplate,Tw, L, alpha,G,epsilon,Tamb,Kt,N,t):
    #Calculate Coefficient for Top of the Plate
        #This independent of Th,Tc
    ha = topForcedConvectionCoefficient(u,Lplate)

    Th, Tc, hw, rho = symbols('Th Tc hw rho')

    sigma = 5.67E-8

    RHS = G*alpha - sigma * epsilon * (Th)**4 - ha * (Th-Tamb) 
    LHS = N * Kt * t * (Th-Tc)
 
    eq1 = Eq(RHS, LHS)
    eq2 = Eq(N*Kt*t * (Th-Tc), hw * (Tc-Tw))


    #Caclulate hw
    water_Amb = water.with_state(
    Input.pressure(101325), Input.temperature(Tw-273)
    )

    rhoAmb = water_Amb.density
    
    #rho = (999.83952 + 16.945176 * (Tc - 273) - 7.9870401E-3 * ((Tc - 273)**2) - 49.170461E-6 * ((Tc - 273)**3) + 105.56302E-9 * ((Tc - 273)**4) - 280.54253E-12 * ((Tc - 273)**5))/(1+16.897850E-3 * (Tc - 273))
    eqn4 = Eq(rho,(999.83952 + 16.945176 * (Tc - 273) - 7.9870401E-3 * ((Tc - 273)**2) - 49.170461E-6 * ((Tc - 273)**3) + 105.56302E-9 * ((Tc - 273)**4) - 280.54253E-12 * ((Tc - 273)**5))/(1+16.897850E-3 * (Tc - 273)))

    Beta = -1/rho * (rhoAmb-rho)/(Tw-Tc)
    
    #Calculate Ralyeigh Number at x = L
    #Find dT
    deltaT = Tc - Tw
    #Define Constants
    g = 9.81
    kinematic_viscosity = 8.005E-7
    #Calculate Ra
    RaL  = abs(g * Beta * deltaT * (L)**3 / (kinematic_viscosity * alpha))
    
    Pr = 6.9
    #Critical Rayleigh Number
    NuL = 0.68 + 0.670 * (RaL) ** (1/4) / (( 1- (0.492/Pr) ** (9/16)) ** (4/9))
    #if RaL < 1E9:
     #   NuL = 0.68 + 0.670 * (RaL) ** (1/4) / (( 1- (0.492/Pr) ** (9/16)) ** (4/9))
   # else:
       # NuL = (0.825+ 0.397 * (RaL) ** (1/6) / (( 1 + (0.492/Pr) ** (9/16)) ** (8/27))**2)

    eqn3 = Eq(hw,NuL* 0.6/L)

    solution = solve((eq1, eq2,eqn3,eqn4), (Th, Tc, hw,rho))
    print(solution)
    


In [15]:
updatedSolution(3,0.5,30+273, 0.5, 0.9,5,0.9,36,1.7,21,3E-3)

NotImplementedError: solving Abs((995.649453937642 + (-16.945176*Tc + 2.8054253e-10*(Tc - 273)**5 - 1.0556302e-7*(Tc - 273)**4 + 4.9170461e-5*(Tc - 273)**3 + 0.0079870401*(Tc - 273)**2 + 3626.193528)/(0.01689785*Tc - 3.61311305))*(0.01689785*Tc - 3.61311305)/(-16.945176*Tc + 2.8054253e-10*(Tc - 273)**5 - 1.0556302e-7*(Tc - 273)**4 + 4.9170461e-5*(Tc - 273)**3 + 0.0079870401*(Tc - 273)**2 + 3626.193528)) when the argument is not real or imaginary.